## Exercise 47 - SparkSQL

- Input: A CSV file containing a list of user profiles
    - Header
    - name,age,gender
    - Each line of the file contains the information about one user
- Output:
    - Select male users (gender=“male”), increase by one their age, and store in the output folder name and age of these users sorted by decreasing age and ascending name (if the age value is the same)
    - the output does not contain the header line
    
- Example of input data:
    - name,age,gender
    - Paul,40,male
    - John,40,male
    - David,15,male
    - Susan,40,female
    - Karen,34,female
- Example of expected output:
    - John,41
    - Paul,41
    - David,16

In [1]:
from pyspark.sql import SparkSession

# Create a Spark Session object
spark = SparkSession.builder.getOrCreate()

# Create a DataFrame from persons_age_name_gender.csv
dfPersons = spark.read.load('./databases/persons_age_name_gender.csv',\
                            format='csv',\
                            header=True,\
                            inferSchema=True)

In [2]:
dfPersons.printSchema()
dfPersons.show()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)

+----------+---+------+
|      Name|Age|Gender|
+----------+---+------+
| Arcangelo| 23|  male|
|  Leonardo| 24|  male|
|Margherita| 10|female|
|  Veronica| 21|female|
|Alessandro| 61|  male|
|   Giorgio| 12|  male|
|      Paul| 40|  male|
|      John| 40|  male|
|     David| 15|  male|
|     Susan| 40|female|
|     Karen| 34|female|
+----------+---+------+



## **Solution 1 - SQL syntax**

In [3]:
# I replace DF name with a easier one
dfPersons.createOrReplaceTempView('persons') 

In [4]:
dfUpdateAgeSorted = spark.sql(\
                             """SELECT name, age+1 as newage
                             FROM persons
                             WHERE gender='male'
                             ORDER BY age desc, name""")

In [5]:
dfUpdateAgeSorted.printSchema()
dfUpdateAgeSorted.show()

root
 |-- name: string (nullable = true)
 |-- newage: integer (nullable = true)

+----------+------+
|      name|newage|
+----------+------+
|Alessandro|    62|
|      John|    41|
|      Paul|    41|
|  Leonardo|    25|
| Arcangelo|    24|
|     David|    16|
|   Giorgio|    13|
+----------+------+



In [6]:
dfUpdateAgeSorted.explain()

== Physical Plan ==
*(2) Project [name#10, newage#29]
+- *(2) Sort [age#11 DESC NULLS LAST, name#10 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(age#11 DESC NULLS LAST, name#10 ASC NULLS FIRST, 200)
      +- *(1) Project [name#10, (age#11 + 1) AS newage#29, age#11]
         +- *(1) Filter (isnotnull(gender#12) && (gender#12 = male))
            +- *(1) FileScan csv [Name#10,Age#11,Gender#12] Batched: false, Format: CSV, Location: InMemoryFileIndex[hdfs://BigDataHA/user/s295406/databases/persons_age_name_gender.csv], PartitionFilters: [], PushedFilters: [IsNotNull(Gender), EqualTo(Gender,male)], ReadSchema: struct<Name:string,Age:int,Gender:string>


## **Solution 2 - SparkSQL API**

In [7]:
dfMales = dfPersons.filter("""gender='male'""")
dfMales.printSchema()
dfMales.show()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)

+----------+---+------+
|      Name|Age|Gender|
+----------+---+------+
| Arcangelo| 23|  male|
|  Leonardo| 24|  male|
|Alessandro| 61|  male|
|   Giorgio| 12|  male|
|      Paul| 40|  male|
|      John| 40|  male|
|     David| 15|  male|
+----------+---+------+



In [8]:
dfUpdatedAge = dfMales.selectExpr('name', 'age+1 as newage')
dfUpdatedAge.printSchema()
dfUpdatedAge.show()

root
 |-- name: string (nullable = true)
 |-- newage: integer (nullable = true)

+----------+------+
|      name|newage|
+----------+------+
| Arcangelo|    24|
|  Leonardo|    25|
|Alessandro|    62|
|   Giorgio|    13|
|      Paul|    41|
|      John|    41|
|     David|    16|
+----------+------+



In [9]:
dfUpdatedAgeSorted = dfUpdatedAge.sort(dfUpdatedAge.newage.desc(), dfUpdatedAge.name)
dfUpdatedAgeSorted.printSchema()
dfUpdatedAgeSorted.show()

root
 |-- name: string (nullable = true)
 |-- newage: integer (nullable = true)

+----------+------+
|      name|newage|
+----------+------+
|Alessandro|    62|
|      John|    41|
|      Paul|    41|
|  Leonardo|    25|
| Arcangelo|    24|
|     David|    16|
|   Giorgio|    13|
+----------+------+

